<a href="https://colab.research.google.com/github/Sarahkh4/AI_Genesis/blob/main/Ai_Genesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# ✅ Install dependencies
!pip install langchain langchain-google-genai pymongo gradio


## Python Learning Path: From Beginner to Intermediate

This learning path is designed to take you from zero Python knowledge to a comfortable intermediate level.  It emphasizes a practical, project-based approach.

**Stage 1: Foundations (Estimated Time: 2-4 weeks)**

* **Learning Goals:**
    * Understand basic Python syntax.
    * Work with data types (integers, floats, strings, booleans).
    * Use operators and control flow (if/else statements, loops).
    * Create and manipulate lists and dictionaries.
    * Understand functions and their usage.
    * Handle basic input/output operations.


* **Resources:**
    * **Book:** "Python Crash Course" by Eric Matthes (available in print and online, some chapters available for free online) – Excellent for beginners, very practical.
    * **Website:**  W3Schools Python Tutorial (w3schools.com/python/) – Concise, well-structured, and free. Good for quick reference.
    * **Course:**  Google's Python Class (developers.google.com/edu/python/

Keyboard interruption in main thread... closing server.


KeyboardInterrupt: 

In [ ]:

# ✅ Set Gemini API key securely (Google Colab specific)
import os
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')  # Ensure you set this in the notebook sidebar > Secrets

# ✅ LangChain & Gemini setup
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate



In [ ]:
# ✅ Gemini model setup
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.5,
    max_retries=2,
    api_key=gemini_api_key
)

# ✅ Prompt Template
prompt_template = PromptTemplate(
    input_variables=["topic"],
    template="""
You are an educational expert. Create a detailed, step-by-step learning path for the topic "{topic}".
Include stages, learning goals, and free resources like books, websites, or courses.
Present it in a clear, structured format.
"""
)



In [ ]:
# ✅ 🔧 MISSING FUNCTION: generate_learning_path()
def generate_learning_path(topic):
    prompt = prompt_template.format(topic=topic)
    try:
        response = llm.invoke(prompt)
        # Extract content properly from Gemini response
        return response.content if hasattr(response, "content") else str(response)
    except Exception as e:
        return f"Error generating learning path: {e}"

# ✅ Optional test to see if Gemini works (can comment out later)
print(generate_learning_path("Python"))



In [ ]:
# ✅ Gradio interface
import gradio as gr

def chatbot(topic):
    if not topic.strip():
        return "Please enter a topic."
    return generate_learning_path(topic)

def download_learning_path(topic):
    plan = generate_learning_path(topic)
    filename = f"{topic}_learning_path.txt"
    with open(filename, "w") as f:
        f.write(plan)
    return filename

with gr.Blocks() as demo:
    gr.Markdown("## 📘 Gemini AI Learning Path Generator")

    with gr.Row():
        topic_input = gr.Textbox(label="Enter a Topic")
        generate_btn = gr.Button("Generate Learning Path")

    output = gr.Textbox(label="Learning Path", lines=20)
    download_btn = gr.Button("Download")

    generate_btn.click(chatbot, inputs=topic_input, outputs=output)
    download_btn.click(download_learning_path, inputs=topic_input, outputs=None)

demo.launch(share=True, debug=True)
